In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import sys
import os

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from src.data.make_dataset import CRCHistoPhenotypes_Patch

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.noise import GaussianNoise


/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Load dataset

In [2]:
dataset = CRCHistoPhenotypes_Patch(patch_size=(27,27))
X, y = dataset.load_patches()

In [3]:
labelencoder = LabelEncoder()
labelencoder.fit(y)
y = labelencoder.transform(y)

In [4]:
len(X)

22444

In [5]:
batch_size = 32
num_classes = 4
epochs = 200
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_CRCPatches_trained_model.h5'

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y, random_state=42)
len_X_train = len(X_train)
X_train = np.concatenate(X_train, axis=0).reshape((len_X_train, )+X[0].shape)

len_X_test = len(X_test)
X_test = np.concatenate(X_test, axis=0).reshape((len_X_test, )+X[0].shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define a Keras Sequential model

In [7]:
model = Sequential()
model.add(GaussianNoise(stddev=0.1, input_shape=(27, 27, 3)))
model.add(Convolution2D(filters=32, kernel_size=(7, 7), activation='relu', input_shape=(27, 27, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Convolution2D(filters=48, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
# model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='tanh'))
# model.add(Dropout(0.25))
model.add(Dense(64, activation='tanh'))
# model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))


In [8]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [9]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(X_test, y_test),
                        workers=4)


Not using data augmentation.
Train on 15037 samples, validate on 7407 samples
Epoch 1/200
15037/15037 [==============================] - 4s 254us/step - loss: 1.3171 - acc: 0.3641 - val_loss: 1.1670 - val_acc: 0.5457
Epoch 2/200
15037/15037 [==============================] - 2s 145us/step - loss: 1.0285 - acc: 0.5840 - val_loss: 0.9704 - val_acc: 0.6046
Epoch 3/200
15037/15037 [==============================] - 2s 121us/step - loss: 0.9037 - acc: 0.6518 - val_loss: 0.8411 - val_acc: 0.6794
Epoch 4/200
15037/15037 [==============================] - 2s 123us/step - loss: 0.8617 - acc: 0.6721 - val_loss: 0.8180 - val_acc: 0.6849
Epoch 5/200
15037/15037 [==============================] - 2s 117us/step - loss: 0.8389 - acc: 0.6795 - val_loss: 0.7944 - val_acc: 0.7004
Epoch 6/200
15037/15037 [==============================] - 2s 116us/step - loss: 0.8267 - acc: 0.6857 - val_loss: 0.8306 - val_acc: 0.6784
Epoch 7/200
15037/15037 [==============================] - 2s 117us/step - loss: 0.8106 

15037/15037 [==============================] - 2s 115us/step - loss: 0.5880 - acc: 0.7817 - val_loss: 0.7211 - val_acc: 0.7378
Epoch 60/200
15037/15037 [==============================] - 2s 113us/step - loss: 0.5888 - acc: 0.7847 - val_loss: 0.8024 - val_acc: 0.7076
Epoch 61/200
15037/15037 [==============================] - 2s 113us/step - loss: 0.5860 - acc: 0.7831 - val_loss: 0.6982 - val_acc: 0.7401
Epoch 62/200
15037/15037 [==============================] - 2s 114us/step - loss: 0.5805 - acc: 0.7862 - val_loss: 0.7254 - val_acc: 0.7343
Epoch 63/200
15037/15037 [==============================] - 2s 114us/step - loss: 0.5758 - acc: 0.7905 - val_loss: 0.6974 - val_acc: 0.7455
Epoch 64/200
15037/15037 [==============================] - 2s 115us/step - loss: 0.5791 - acc: 0.7863 - val_loss: 0.6970 - val_acc: 0.7428
Epoch 65/200
15037/15037 [==============================] - 2s 114us/step - loss: 0.5731 - acc: 0.7875 - val_loss: 0.7951 - val_acc: 0.7213
Epoch 66/200
15037/15037 [=======

KeyboardInterrupt: 

In [19]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [20]:
score = model.evaluate(X_test, y_test, verbose=0)
score

[0.72940465018899558, 0.74915620372799774]